In [81]:
# soft version

import numpy as np
import matplotlib.pyplot as plt


m = 20 # number of devices
n = 5  # number of servers

# f(x, y, mu)
def f(x, y, mu):
    f_out = np.sum(x_t[:,1:]*mu)
    return f_out

# min f(x, y. mu)
def oracle(y, mu):
    x = np.ones((m, n+1))
    x /= (n+1)
    return x

N = 10 # number of experiments
T = 1000 # rounds per experiment
reg = np.zeros((N,T))

for u in range(N):
    mu = np.random.rand(m,n) # real mu
    mu_hat = np.zeros_like(mu) # empirical mean
    T_ij = np.ones_like(mu) # total number of times arm (i,j) is played
    for t in range(T):
        y = 100*np.random.rand(m)
        x_opt = oracle(y, mu)
        f_opt = f(x_opt, y, mu)
        
        rho_ij = np.sqrt(3*np.log(t+1)/(2*T_ij))
        mu_bar = np.max(mu_hat - rho_ij, 0) # LCB
        
        x_t = oracle(y, mu_bar)
        f_t = f(x_t, y, mu)
        
        # sample j based on x_t[i], observe c_ij, update mu_hat[i,j]        
        for i in range(m):
            j = np.random.choice(n+1, p=x_t[i])
            if j != 0:
                j -= 1
                c_ij = int(np.random.rand() < mu[i,j])
                T_ij[i, j] += 1
                mu_hat[i, j] += (c_ij - mu_hat[i, j]) / T_ij[i, j]

        # calculate regert
        reg[u, t] = f_t - f_opt      